# Retrieval-Augmented Generation (RAG) with OpenAI File Search
 
#### This notebook demonstrates how to implement a Retrieval-Augmented Generation (RAG) workflow using OpenAI's Assistants API and File Search capabilities. It guides you through uploading knowledge base documents, configuring the assistant, and performing question answering over your custom data.

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
# Set OpenAI key in the environment
import os

api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [4]:
from openai import OpenAI # must be above 1.66
from IPython.display import Markdown

client = OpenAI()

## Create Knowledge Component

Prepare and upload relevant documents to be used as a knowledge base for the assistant.

In [5]:
# Define the files needed
files_path = ["./resources/Termos e Condições _ Politica de Privacidade da Bitte (PT_EN).docx",
              "./resources/Bitte - Apresentação Corporativa.pdf"]
file_ids = []

# Upload the files to the API
for file_path in files_path:
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file = file_content,
        # The 'purpose' parameter specifies how the uploaded file will be used by the OpenAI API.
        # Setting purpose="assistants" means the file is intended for use with the Assistants API,
        # such as for retrieval-augmented generation (RAG) or knowledge base augmentation.
        purpose = "assistants"
    )
    # Save the File ID
    file_ids.append(result.id)

print(file_ids)

['file-2kciqvqat4xaqPuKwxjLdj', 'file-M2M74EEEXWyk8zpFAJWLae']


## Vector Store
 
In the next two cells, we are creating a **vector store** to enable efficient and accurate retrieval of information from our uploaded documents. Here’s a structured explanation:

**1. Purpose of Creating a Vector Store**
- A vector store is a specialized database that stores document embeddings (numerical representations of text).
- By converting documents into vectors, we enable the system to perform semantic search, finding relevant information based on meaning rather than just keywords.

**2. What We Will Do**
- **Create a Vector Store:** We initialize a new vector store using the OpenAI API.
- **Add Files:** We upload our knowledge base documents (e.g., terms, conditions, and presentations) to the vector store, where they are processed and indexed.

**3. How This Helps**
- When a user asks a question, the system can search the vector store to find the most relevant sections of the documents.
- This process ensures that the assistant can provide accurate, contextually relevant answers by referencing the actual content of the uploaded files.
- It forms the foundation for Retrieval-Augmented Generation (RAG), where the language model augments its responses with information retrieved from the knowledge base.

**Summary Table**
| Step                | Action                                      | Benefit                                      |
|---------------------|---------------------------------------------|----------------------------------------------|
| Create Vector Store | Initialize storage for document embeddings  | Enables semantic search                      |
| Add Files           | Upload and index knowledge documents        | Makes information retrievable for the model  |

By structuring our knowledge in a vector store, we empower the assistant to deliver more precise and trustworthy answers based on our specific documents.

In [6]:
# Create a Vector Store
vector_store = client.vector_stores.create(
    name = "Bitte Vector Store",
)
print(vector_store.id)

vs_6898aed613808191b666e803ad446d5f


In [7]:
# Add files to the vector store
for file_id in file_ids:
  result = client.vector_stores.files.create(
      vector_store_id = vector_store.id,
      file_id = file_id,
  )

# RAG System

 
## **Building the RAG System with OpenAI File Search**

In the next cell, we construct a Retrieval-Augmented Generation (RAG) system using the OpenAI API. Here’s a breakdown of the parameters used in the `client.responses.create` method:

| Parameter   | Description                                                                                                    |
|-------------|----------------------------------------------------------------------------------------------------------------|
| `model`     | Specifies the language model to use. In this case, `"gpt-4.1-mini"` is selected for generating responses.      |
| `input`     | The user’s query or prompt. Here, it asks: `"List the benefits of Bitte"`.                                     |
| `tools`     | A list of tools to augment the model. This includes:                                                           |
|             | - `type`: `"file_search"` enables searching over uploaded documents.                                            |
|             | - `vector_store_ids`: A list containing the ID(s) of the vector store(s) to search. Here, `[vector_store.id]`. |
|             | - `max_num_results`: Limits the number of search results returned (set to 20).                                  |
| `include`   | Specifies additional information to include in the response. Here, `["file_search_call.results"]` ensures that |
|             | the retrieved document references are included alongside the model’s answer.                                    |

**Summary:**  
This setup allows the language model to answer questions by searching the uploaded knowledge base (vector store) and referencing relevant documents, resulting in more accurate and context-aware responses.


In [8]:
# Build the RAG system
response = client.responses.create(
    model = "gpt-4.1-mini",
    input = "List the benefits of Bitte",
    tools = [{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 20
    }],
    include = ["file_search_call.results"] # Include the references
)

In [9]:
# What GPT queried the Vector Store
response.output[0].queries

['benefits of Bitte']

In [10]:
# What was the context retrieved?
response.output[0].results[19].text

'4.2.2 O utilizador aceita que se pretender alterar ou cancelar o seu pedido/encomenda, após finalização e confirmação do respectivo pagamento, deverá contactar o serviço de apoio/suporte ao cliente do “BIIPⓇ”, que entrará em contacto com o restaurante parceiro em causa, no sentido de comunicar as alterações solicitadas. Todavia, o utilizador reconhece ainda que o “BIIPⓇ” pode não conseguir contactar com o restaurante e que não assume qualquer responsabilidade nem de qualquer modo garante que o restaurante parceiro, após ter dado início à preparação do pedido/encomenda do utilizador, aceitará as alterações solicitadas. 4.3. O utilizador reconhece que quando o pagamento por si efetuado não for autorizado pela instituição bancária correspondente, terá de efetuar o pagamento através de um método alternativo ou o seu pedido/encomenda não será processado nem comunicado pela “BIIPⓇ” ao restaurante parceiro selecionado pelo utilizador. 4.4. O utilizador aceita que após a receção do seu pedido

In [11]:
# What is the answer?
Markdown(response.output[1].content[0].text)

The benefits of Bitte, a digital solution for restaurants, are as follows:

1. **For Customers:**
   - Improved Customer Experience: Customers can better understand the menu with photos and detailed descriptions, helping them make choices that match their preferences.
   - Personal Interaction: Digital menus do not reduce interaction but allow for more personalized, attentive, and careful service by the staff.
   - Convenience: Customers can use their smartphones to order, pay, make reservations, or request takeaway and delivery services with ease.

2. **For Restaurants:**
   - Simplicity and Ease of Use: The app allows easy menu management for the restaurant and simple viewing for customers.
   - Staff Efficiency: The tool supports the staff, freeing up their time to provide more personalized service.
   - Increased Sales and Average Ticket Size: Studies show that digital menus with attractive images increase sales.
   - Automation and Efficiency Gains (Premium version): Automating processes can reduce waiting times and optimize restaurant management, addressing challenges like staff recruitment and retention.
   - Data-Driven Decision Making: Access to metrics such as popular and profitable dishes, service speed, and sales variations helps strategic decisions.
   - More Sales Channels: Expands offerings to include takeaway and delivery with digital ordering.
   - Digitalization and Innovation: Helps restaurants keep up with market demands, reach discerning customers, and enhance offers with digital tools.
   - Easy to Start: No need for programming knowledge, software installation, or spending time creating the menu from scratch as Bitte converts menus from PDFs using OCR.

3. **Additional Features in Premium:**
   - Multi-language Menu Translation.
   - Cross-selling features to suggest complementary items (e.g., wine with a dish) to increase the average spend.
   - Table reservation integration with Google.
   - Campaign creation and promotions.
   - Detailed analytics on profitability and popularity of menu items.

4. **Business Impact:**
   - Restaurants using digital menus see an average revenue increase of 10-20%.
   - Enhanced customer loyalty by offering a modern and personalized service experience.
   - Provides a competitor edge through digital transformation.

In summary, Bitte supports restaurants by digitalizing and improving the customer experience, increasing operational efficiency, expanding sales channels, and enabling data-driven management, all with an easy-to-adopt platform that offers both free and premium services tailored to the restaurant's needs.

# RAG System with Developper Messages

In [12]:
# Build the RAG system
response = client.responses.create(
    model = "gpt-4.1-mini",
    input = "List the benefits of Bitte",
    instructions = "Answer like a funny boss who prefers terms like pre-revenue and cash burn ratio",
    tools = [{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        "max_num_results": 20
    }],
    include = ["file_search_call.results"] # Include the references
)

In [13]:
# What is the answer?
Markdown(response.output[1].content[0].text)

Alright team, gather ’round! Let me give you the skinny on all the benefits of Bitte—the slick digital sidekick for your restaurant that’s here to turn your pre-revenue hustle into serious cash flow, trimming that pesky cash burn ratio.

Here’s why Bitte will make you want to shout “Yes, bitte!”:

1. **Digital Menu Magic**  
   Your customers get to scroll through a beautiful digital menu on their smartphones. They can check out dish pics, make savvy choices, and BAM! – better experience equals happier diners who come back for more.

2. **Freemium to Premium – Scale Like a Pro**  
   Start free, baby! Use the digital menu, create campaigns, add multi-language translations, and cross-sell to boost that average ticket size. When you’re ready to crank it up, go Premium for table reservations, mobile order & payments, takeaway, delivery, and deep analytics on profitability and popularity.

3. **Staff Efficiency Boost**  
   Digital tools mean your team spends less time on grunt work and more time wowing customers. More happy staff, less turnover. That’s like lowering your cash burn ratio by doing more with less hassle!

4. **AI-Driven Smarts for Better Management**  
   Bitte’s intelligence looks at your data vs. the best in the biz and tells you concrete actions to improve operations. Know your most popular and most profitable dishes, speed of service, and when your kitchen is cranking or slacking.

5. **Retention & Sales Growth – Cha-Ching!**  
   Studies show restaurants with digital menus get a 10-20% revenue lift! With tailored pairing suggestions (think: that fancy wine that has a fat margin or a dessert that seals the deal), your ticket value shoots upwards.

6. **More Channels, More Money**  
   Takeaway, delivery, in-house orders — it’s all seamless. Your customers get what they want, you get more sales without extra headaches.  

7. **No Tech Headaches**  
   No need to be a coding ninja or install apps everywhere. Upload your menu PDF, get a QR code, and you’re open for business digitally faster than you can say “burn rate.”

8. **Data-Driven Decisions**  
   Actionable insights mean you stop guessing and start profiting. Optimize your menu and service based on real customer data — because gut feelings don't pay the rent.

9. **Easy Setup & Support**  
   Three steps: sign up, send in your menu PDF, boom—you get a QR code. Plus, customer support is just a WhatsApp or call away.

So, in the lingo of a funny boss who's all about numbers and growth: Bitte is your crystal ball and power tool rolled into one, reducing your pre-revenue pains, managing your cash burn, and helping you turn every customer interaction into a profit multiplier. No more faffing about with paper menus or staff burnout — it’s digital efficiency with a side of sales uplift!

If you want me to break down how the free vs premium features stack up or the exact workflow, just shout! But basically, Bitte turns your restaurant into a smooth-running, money-making digital champ.